In [3]:
%pip install -qU langchain langgraph langchain-openai langchain-community langchain-text-splitters chromadb pypdf


Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langchain.agents.react.agent import create_react_agent
from langchain.agents import AgentExecutor
from claim_verifier import make_search_tool, SYSTEM_PROMPT

In [18]:
import sys
from pathlib import Path

sys.path.append(str(Path().resolve()))

from claim_verifier import build_retriever, make_search_tool, SYSTEM_PROMPT


In [10]:
import os

In [ ]:
OPENAI_API_KEY = "<OPENAI_KEY>"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [15]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [20]:
PDF_PATH = "data/attention_all_u_need.pdf"

retriever = build_retriever(
    pdf_path=PDF_PATH,
    persist_dir="chroma_claim_db",
    collection_name="claim_pdf",
    k=5,
)

claim_tool = make_search_tool(retriever)  


In [33]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("system", "You have access to the following tools:\n\n{tools}"),
    ("system", "Tool names: {tool_names}"),
    ("system", """
When you use tools, you MUST respond in this exact format:

Thought: <short reasoning>
Action: <one of {tool_names}>
Action Input: <string query>

When you are ready to answer the user, you MUST respond in this exact format:

Final: <answer with citations like (page X). If not in PDF, write: Not found in the provided PDF.>
""".strip()),
    ("human", "{input}"),
    ("assistant", "{agent_scratchpad}"),
])


In [34]:


model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent = create_react_agent(
    llm=model,
    tools=[claim_tool],
    prompt=prompt,
)

executor = AgentExecutor(
    agent=agent,
    tools=[claim_tool],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=6,
)


In [35]:


res = executor.invoke({"input": "tell me is this paper really about transformer?"})
print(res["output"])




> Entering new AgentExecutor chain...
Thought: I need to check if the PDF mentions anything about transformers to confirm the topic of the paper. 
Action: claim_verifier
Action Input: "transformer" 

Final: Not found in the provided PDF.[source=data/attention_all_u_need.pdf page=9] Table 4: The Transformer generalizes well to English constituency parsing (Results are on Section 23 of WSJ) Parser Training WSJ 23 F1 Vinyals & Kaiser el al. (2014) [37] WSJ only, discriminative 88.3 Petrov et al. (2006) [29] WSJ only, discriminative 90.4 Zhu et al. (2013) [40] WSJ only, discriminative 90.4 Dyer et al. (2016) [8] WSJ only, discriminative 91.7 Transformer (4 layers) WSJ only, discriminative 91.3 Zhu et al. (2013) [40] semi-supervised 91.3 Huang & Harper (2009) [14] semi-supervised 91.3 McClosky et al. (2006) [26] semi-supervised 92.1 Vinyals & Kaiser el al. (2014) [37] semi-supervised 92.1 Transformer (4 layers) semi-supervised 92.7 Luong et al. (2015) [23] multi-task 93.0 Dyer et al. (201

/Users/satyammittal/github/AI-ML/demo_applications/agent-patterns-pack/claim_verifier.py:91: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  hits = retriever.get_relevant_documents(query)


Final: The paper is indeed about the Transformer, as it discusses the Transformer model as a sequence transduction model based entirely on attention, replacing recurrent layers with multi-headed self-attention (page 9). It also presents results comparing the Transformer to other models in translation tasks, indicating its effectiveness and efficiency (page 7).Invalid Format: Missing 'Action:' after 'Thought:'Thought: I need to check if the PDF mentions anything about transformers to confirm the topic of the paper. 
Action: claim_verifier
Action Input: "transformer" 

Final: The paper is indeed about the Transformer, as it discusses the Transformer model as a sequence transduction model based entirely on attention, replacing recurrent layers with multi-headed self-attention (page 9). It also presents results comparing the Transformer to other models in translation tasks, indicating its effectiveness and efficiency (page 7).[source=data/attention_all_u_need.pdf page=1] language modeling 